In [0]:
!pip install keras_metrics
!apt-get -qq install -y graphviz && pip install -q py
!pip install pydot

In [0]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate,StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import keras_metrics
from keras.utils import plot_model
from keras.datasets import cifar10
from sklearn.metrics import f1_score
import tensorflow as tf


In [0]:
X = pd.read_csv("drive/My Drive/Colab Notebooks/X.csv",sep = ' ', header=None,dtype=float)
X = X.values

y = pd.read_csv("drive/My Drive/Colab Notebooks/y_williams_vs_others.csv",sep = ' ', header=None,dtype=float)
y_bush = y.values.ravel()

In [0]:
print("X shape", X.shape)
print('Y William shape', y_bush.shape)

X shape (13233, 4096)
Y William shape (13233,)


In [0]:
batch_size = 100
num_classes = 2
epochs = 30

# input image dimensions
# img_rows, img_cols = 64, 64

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y_bush, test_size=1. / 3, random_state=8854, shuffle = True, stratify = y_bush)

In [0]:
x_train = x_train.reshape(-1,64,64,1)
x_test = x_test.reshape(-1,64,64,1)

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

X_grey_train=tf.image.rgb_to_grayscale(x_train,name=None)
X_grey_test=tf.image.rgb_to_grayscale(x_test,name=None)

X_reshaped_train = tf.image.resize_images(X_grey_train,(64,64))
X_reshaped_test = tf.image.resize_images(X_grey_test,(64,64))

y_train_binary=list()
for i in range(len(y_train)):
 if y_train[i]==5:
   y_train_binary.append(1)
 elif y_train[i]!=5:
   y_train_binary.append(0)

In [0]:
y_test_binary=list()
for i in range(len(y_test)):
 if y_test[i]==5:
   y_test_binary.append(1)
 elif y_test[i]!=5:
   y_test_binary.append(0)	


In [0]:
y_train_binary_array=np.asanyarray(y_train_binary)
type(y_train_binary_array)

y_test_binary_array=np.asanyarray(y_test_binary)
type(y_test_binary_array)

init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
X_reshaped_train=X_reshaped_train.eval(session=sess)
X_reshaped_test=X_reshaped_test.eval(session=sess)

In [0]:
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(64,64,1)))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr = 0.0001), metrics=['accuracy'])


In [0]:
model.fit(X_reshaped_train, y_train_binary_array,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_reshaped_train, y_train_binary_array))

Train on 50000 samples, validate on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 21s 428us/step - loss: 0.2895 - acc: 0.9001 - val_loss: 0.2655 - val_acc: 0.9034
Epoch 2/30
50000/50000 [==============================] - 21s 413us/step - loss: 0.2453 - acc: 0.9083 - val_loss: 0.2334 - val_acc: 0.9116
Epoch 3/30
50000/50000 [==============================] - 21s 415us/step - loss: 0.2273 - acc: 0.9146 - val_loss: 0.2142 - val_acc: 0.9190
Epoch 4/30
50000/50000 [==============================] - 21s 413us/step - loss: 0.2159 - acc: 0.9190 - val_loss: 0.2016 - val_acc: 0.9236
Epoch 5/30
50000/50000 [==============================] - 21s 413us/step - loss: 0.2060 - acc: 0.9221 - val_loss: 0.2143 - val_acc: 0.9174
Epoch 6/30
50000/50000 [==============================] - 21s 413us/step - loss: 0.1958 - acc: 0.9264 - val_loss: 0.1842 - val_acc: 0.9308
Epoch 7/30
50000/50000 [==============================] - 21s 413us/step - loss: 0.1882 - acc: 0.9296 - val_loss: 0.

In [0]:
s = model.predict_classes(X_reshaped_test)
fscore = f1_score(y_test_binary_array,s)
print(fscore)

0.5501519756838906


In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 8822 samples, validate on 4411 samples
Epoch 1/30
8822/8822 [==============================] - 3s 393us/step - loss: 0.0263 - acc: 0.9960 - val_loss: 0.0223 - val_acc: 0.9961
Epoch 2/30
8822/8822 [==============================] - 3s 382us/step - loss: 0.0222 - acc: 0.9960 - val_loss: 0.0207 - val_acc: 0.9961
Epoch 3/30
8822/8822 [==============================] - 3s 377us/step - loss: 0.0213 - acc: 0.9960 - val_loss: 0.0193 - val_acc: 0.9961
Epoch 4/30
8822/8822 [==============================] - 3s 376us/step - loss: 0.0190 - acc: 0.9960 - val_loss: 0.0198 - val_acc: 0.9961
Epoch 5/30
8822/8822 [==============================] - 3s 381us/step - loss: 0.0180 - acc: 0.9960 - val_loss: 0.0178 - val_acc: 0.9961
Epoch 6/30
8822/8822 [==============================] - 3s 377us/step - loss: 0.0177 - acc: 0.9960 - val_loss: 0.0170 - val_acc: 0.9961
Epoch 7/30
8822/8822 [==============================] - 3s 379us/step - loss: 0.0160 - acc: 0.9960 - val_loss: 0.0168 - val_acc: 0.9961


In [0]:

s = model.predict_classes(x_train)
fscore_train = f1_score(y_train,s)
print('train f1 = ', fscore_train)

s = model.predict_classes(x_test)
fscore_test = f1_score(y_test,s)
print('test f1 = ', fscore_test)

train f1 =  0.8333333333333333
test f1 =  0.48
